<a href="https://colab.research.google.com/github/sumitrathore111/main.py/blob/main/face_detection_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 88.9 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.0
    Uninstalling tensorboard-2.17.0:
      Successfully uninstalled tensorboard-2.17.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.0
    Uninstalling tensorflow-2.17.0:
      Successfully uninstalled tensorflow-2.17.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompat

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Lambda, Dense, Flatten, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import cv2
import os
from google.colab import files
from sklearn.model_selection import train_test_split



In [3]:
# Step 1: Create Folders for Images (Sumit, Suraj, Ashis, Shivam)
import os
from google.colab import files
base_dir = '/content/faces'
os.makedirs(base_dir, exist_ok=True)
os.makedirs(os.path.join(base_dir, 'Sumit Rathore'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'Suraj Singh'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'Ashis Gautam'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'Shivam Pandey'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'Recognition'), exist_ok=True)  # Create Recognition folder
# Function to upload images to the respective folders
def upload_images(folder_name):
    uploaded_files = files.upload()
    for file_name in uploaded_files.keys():
        # Move uploaded file to the desired folder
        os.rename(file_name, os.path.join(base_dir, folder_name, file_name))
    print(f"Uploaded images to {folder_name} folder.")

print("Upload images for Sumit Rathore:")
upload_images('Sumit Rathore')
print("Upload images for Suraj Singh:")
upload_images('Suraj Singh')
print("Upload images for Ashis Gautam:")
upload_images('Ashis Gautam')
print("Upload images for Shivam Pandey:")
upload_images('Shivam Pandey')

# Upload new images for recognition
print("Upload new images for recognition:")
upload_images('Recognition')


Upload images for Sumit Rathore:


Saving sumit 8.jfif to sumit 8.jfif
Saving sumit 7.jfif to sumit 7.jfif
Saving sumit 6.jfif to sumit 6.jfif
Saving sumit 5.jfif to sumit 5.jfif
Saving sumit 4.jfif to sumit 4.jfif
Saving sumit 3.jfif to sumit 3.jfif
Saving sumit 2.jfif to sumit 2.jfif
Saving Sumit 1.jfif to Sumit 1.jfif
Uploaded images to Sumit Rathore folder.
Upload images for Suraj Singh:


Saving suraj 6.jfif to suraj 6.jfif
Saving suraj 5.jfif to suraj 5.jfif
Saving suraj 4.jfif to suraj 4.jfif
Saving suraj 3.jfif to suraj 3.jfif
Saving suraj 2.jfif to suraj 2.jfif
Saving suraj 1.jfif to suraj 1.jfif
Uploaded images to Suraj Singh folder.
Upload images for Ashis Gautam:


Saving ashis 5.jfif to ashis 5.jfif
Saving ashis 4.jfif to ashis 4.jfif
Saving ashis 3.jfif to ashis 3.jfif
Saving ashis 2.jfif to ashis 2.jfif
Saving Ashish 1.jfif to Ashish 1.jfif
Uploaded images to Ashis Gautam folder.
Upload images for Shivam Pandey:


Saving shivam 9.jfif to shivam 9.jfif
Saving SHIVAM 8.jfif to SHIVAM 8.jfif
Saving SHIVAM 7.jfif to SHIVAM 7.jfif
Saving shivam 6.jfif to shivam 6.jfif
Saving shivam 5.jfif to shivam 5.jfif
Saving shivam 4.jfif to shivam 4.jfif
Saving shivam 3.jfif to shivam 3.jfif
Saving shivam 2.jfif to shivam 2.jfif
Saving shivam 1.jfif to shivam 1.jfif
Uploaded images to Shivam Pandey folder.
Upload new images for recognition:


Saving test  image 2.jfif to test  image 2.jfif
Uploaded images to Recognition folder.


In [15]:
# Step 2: Load and Preprocess Data
import numpy as np
import cv2
import os

def preprocess_image(image_path, target_size=(105, 105)):
    # Check if the image file exists before attempting to read it
    if not os.path.exists(image_path):
        print(f"Warning: Image file not found: {image_path}")
        return None  # Return None to indicate failure

    image = cv2.imread(image_path)

    # Check if the image was successfully loaded
    if image is None:
        print(f"Warning: Failed to load image: {image_path}")
        return None  # Return None to indicate failure

    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    image = cv2.resize(image, target_size)  # Resize to target size
    image = image / 255.0  # Normalize the image
    return image

def load_data(base_dir):
    pairs = []
    labels = []

    # Load images for all four people
    sumit_images = [os.path.join(base_dir, 'Sumit Rathore', img) for img in os.listdir(os.path.join(base_dir, 'Sumit Rathore'))]
    suraj_images = [os.path.join(base_dir, 'Suraj Singh', img) for img in os.listdir(os.path.join(base_dir, 'Suraj Singh'))]
    ashis_images = [os.path.join(base_dir, 'Ashis Gautam', img) for img in os.listdir(os.path.join(base_dir, 'Ashis Gautam'))]
    shivam_images = [os.path.join(base_dir, 'Shivam Pandey', img) for img in os.listdir(os.path.join(base_dir, 'Shivam Pandey'))]

    # Create pairs of similar (1) and dissimilar (0) images for each pair of persons
    # Similar pairs
    for img1 in sumit_images:
        for img2 in sumit_images:
            if img1 != img2:
                pairs.append([img1, img2])
                labels.append(1)  # Similar images
    for img1 in suraj_images:
        for img2 in suraj_images:
            if img1 != img2:
                pairs.append([img1, img2])
                labels.append(1)  # Similar images
    for img1 in ashis_images:
        for img2 in ashis_images:
            if img1 != img2:
                pairs.append([img1, img2])
                labels.append(1)  # Similar images
    for img1 in shivam_images:
        for img2 in shivam_images:
            if img1 != img2:
                pairs.append([img1, img2])
                labels.append(1)  # Similar images

    # Dissimilar pairs
    for img1 in sumit_images:
        for img2 in suraj_images + ashis_images + shivam_images:
            pairs.append([img1, img2])
            labels.append(0)  # Dissimilar images
    for img1 in suraj_images:
        for img2 in sumit_images + ashis_images + shivam_images:
            pairs.append([img1, img2])
            labels.append(0)  # Dissimilar images
    for img1 in ashis_images:
        for img2 in sumit_images + suraj_images + shivam_images:
            pairs.append([img1, img2])
            labels.append(0)  # Dissimilar images
    for img1 in shivam_images:
        for img2 in sumit_images + suraj_images + ashis_images:
            pairs.append([img1, img2])
            labels.append(0)  # Dissimilar images

    return np.array(pairs), np.array(labels)

pairs, labels = load_data(base_dir)


In [16]:
import tensorflow as tf  # Ensure tensorflow is imported
from tensorflow.keras.layers import Input, Conv2D, Lambda, Dense, Flatten, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import cv2
import os
from google.colab import files
from sklearn.model_selection import train_test_split

# Step 1: Define the base network
def create_base_network(input_shape):
    inputs = Input(shape=input_shape)  # Define input layer
    x = Conv2D(64, (10, 10), activation='relu')(inputs)  # Conv layer 1
    x = MaxPooling2D()(x)  # Maxpool 1
    x = Conv2D(128, (7, 7), activation='relu')(x)  # Conv layer 2
    x = MaxPooling2D()(x)  # Maxpool 2
    x = Conv2D(128, (4, 4), activation='relu')(x)  # Conv layer 3
    x = MaxPooling2D()(x)  # Maxpool 3
    x = Conv2D(256, (4, 4), activation='relu')(x)  # Conv layer 4
    x = Flatten()(x)  # Flatten the output
    x = Dense(4096, activation='sigmoid')(x)  # Dense layer
    return Model(inputs, x)  # Return the base network

# Step 2: Define the Siamese Network
def create_siamese_network(input_shape):
    base_network = create_base_network(input_shape)

    # Inputs for both images in the pair
    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)

    # Process both images through the base network
    processed_a = base_network(input_a)
    processed_b = base_network(input_b)

    # Compute the absolute difference between the embeddings of the two images
    distance = Lambda(lambda embeddings: K.abs(embeddings[0] - embeddings[1]))([processed_a, processed_b])

    # Output layer: Sigmoid to output similarity score
    outputs = Dense(1, activation='sigmoid')(distance)

    # Create the Siamese model
    siamese_network = Model([input_a, input_b], outputs)

    return siamese_network

# Define the input shape
input_shape = (105, 105, 1)  # Grayscale images with size 105x105

# Create the Siamese Network
siamese_network = create_siamese_network(input_shape)

# Print the summary of the model
siamese_network.summary()



Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 105, 105, 1)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_5             │ (None, 105, 105, 1)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_2 (Functional) │ (None, 4096)           │     38,947,648 │ input_layer_4[0][0],   │
│                           │                        │                │ input_layer_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_1 (Lambda)         │ (None, 4096)           │              0 │ functional_2[0][0],    │
│                           │                        │                │ functional_2[1][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 1)              │          4,097 │ lambda_1[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 38,951,745 (148.59 MB)

 Trainable params: 38,951,745 (148.59 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# Step 4: Compile and Train the Model
siamese_network.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Split dataset and preprocess pairs for training
pairs_train, pairs_test, labels_train, labels_test = train_test_split(pairs, labels, test_size=0.2, random_state=42)



In [18]:
# Preprocess pairs for training
pairs_train = np.array([[preprocess_image(pair[0]), preprocess_image(pair[1])] for pair in pairs_train])
pairs_test = np.array([[preprocess_image(pair[0]), preprocess_image(pair[1])] for pair in pairs_test])

# Reshape the input data to match the expected shape of the Siamese network
pairs_train_a = pairs_train[:, 0].reshape(-1, 105, 105, 1) #Reshape the first image in the pair
pairs_train_b = pairs_train[:, 1].reshape(-1, 105, 105, 1) #Reshape the second image in the pair
pairs_test_a = pairs_test[:, 0].reshape(-1, 105, 105, 1) #Reshape the first image in the pair
pairs_test_b = pairs_test[:, 1].reshape(-1, 105, 105, 1) #Reshape the second image in the pair


In [19]:
 #Train the model using the reshaped input data
siamese_network.fit([pairs_train_a, pairs_train_b], labels_train, batch_size=32, epochs=10, validation_data=([pairs_test_a, pairs_test_b], labels_test))


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_29', 'keras_tensor_30']. Received: the structure of inputs=('*', '*')
  warnings.warn(


29/29 ━━━━━━━━━━━━━━━━━━━━ 8s 180ms/step - accuracy: 0.7081 - loss: 0.5212 - val_accuracy: 0.8222 - val_loss: 0.3830
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 62ms/step - accuracy: 0.8850 - loss: 0.2701 - val_accuracy: 0.9333 - val_loss: 0.1454
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - accuracy: 0.9688 - loss: 0.0727 - val_accuracy: 0.9778 - val_loss: 0.0431
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 60ms/step - accuracy: 1.0000 - loss: 0.0108 - val_accuracy: 1.0000 - val_loss: 0.0018
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 64ms/step - accuracy: 1.0000 - loss: 0.0018 - val_accuracy: 1.0000 - val_loss: 0.0015
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - accuracy: 1.0000 - loss: 0.0021 - val_accuracy: 1.0000 - val_loss: 3.6901e-04
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - accuracy: 1.0000 - loss: 0.0019 - val_accuracy: 1.0000 - val_loss: 2.4265e-04
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 60ms/step - accuracy: 1.0000 - loss: 8.2199e-04 - val_accuracy: 1.0000 -

In [25]:
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K

# Function to preprocess the input image (same as used during training)
def preprocess_input_image(image_path, target_size=(105, 105)):
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image not found: {image_path}")

    image = cv2.imread(image_path)

    if image is None:  # Check if the image could not be loaded
        raise ValueError(f"Failed to load image from path: {image_path}")

    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    image = cv2.resize(image, target_size)  # Resize to target size
    image = image / 255.0  # Normalize the image
    return image

# Define the prediction function with thresholding for "Unknown"
def recognize_face(model, input_image_path, known_images_folder, threshold=0.7):
    """
    Recognize the face in the input image by comparing it with known images.
    Returns the best match or "Unknown" if no match exceeds the threshold.
    """
    # Preprocess the input image
    input_image = preprocess_input_image(input_image_path)

    # Reshape to match model input (1, 105, 105, 1)
    input_image_reshaped = input_image.reshape(1, 105, 105, 1)

    # List to hold results (image name and similarity score)
    results = []

    # Iterate over known images to compare
    for known_image in os.listdir(known_images_folder):
        known_image_path = os.path.join(known_images_folder, known_image)
        # Preprocess the known image
        known_image_processed = preprocess_input_image(known_image_path)
        known_image_reshaped = known_image_processed.reshape(1, 105, 105, 1)

        # Predict using the Siamese network
        prediction = model.predict([input_image_reshaped, known_image_reshaped])
        similarity_score = prediction[0][0]  # Similarity score from the model

        # Append the result
        results.append((known_image, similarity_score))

    # Find the best match based on the highest similarity score
    best_match = max(results, key=lambda x: x[1])

    # Check if the best match exceeds the threshold
    if best_match[1] >= threshold:
        return best_match  # Return the best match and similarity score
    else:
        return ("Unknown", best_match[1])  # Return "Unknown" if no match found

# Example usage:
def find_best_match(input_image_path, known_images_folders, model, threshold=0.7):
    """
    Compare the input image to images in multiple known folders and return the best match.
    """
    try:
        results = []

        # Iterate over the known image folders and predict for each folder
        for folder in known_images_folders:
            result = recognize_face(model, input_image_path, folder, threshold)
            results.append(result)

        # Find the best match across all folders
        best_match = max(results, key=lambda x: x[1])

        # Output the result
        if best_match[0] == "Unknown":
            print("The person in the photo is Unknown.")
        else:
            print(f"Best match: {best_match[0]} with similarity score: {best_match[1]:.4f}")

    except FileNotFoundError as e:
        print(e)
    except ValueError as e:
        print(e)


# Define the input image path for recognition
input_image_path = '/content/faces/Recognition/asish test 2.jfif'  # Replace with your test image path

# List of known image folders
known_images_folder_sumit = '/content/faces/Sumit Rathore'
known_images_folder_suraj = '/content/faces/Suraj Singh'
known_images_folder_ashis = '/content/faces/Ashis Gautam'
known_images_folder_shivam = '/content/faces/Shivam Pandey'

# List of known image folders to compare with the input image
known_images_folders = [known_images_folder_sumit, known_images_folder_suraj, known_images_folder_ashis, known_images_folder_shivam]

# Call the function to find the best match
find_best_match(input_image_path, known_images_folders, siamese_network, threshold=0.9)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━